# Монте-Карло: практика

In [2]:
import pandas as pd
import numpy as np

In [3]:
dice = pd.Series([1, 2, 3, 4, 5, 6])

In [13]:
dice.sample(1).values[0]

6

In [19]:
rolls = [dice.sample(1).values[0] for _ in range(100000)]

In [20]:
rolls[:10]

[2, 2, 4, 1, 5, 1, 6, 6, 6, 3]

In [21]:
pd.Series(rolls).value_counts(normalize=True)

5    0.17057
2    0.16693
4    0.16656
6    0.16545
3    0.16544
1    0.16505
dtype: float64

In [22]:
dice.value_counts(normalize=True)

1    0.166667
2    0.166667
3    0.166667
4    0.166667
5    0.166667
6    0.166667
dtype: float64

# *Парадокс дней рождений (никогда не равно 100%)*

Определить вероятность того, что в группе, состоящей из 23 человек, у двух людей
будет совпадение дней рождений (число и месяц)

In [23]:
bd = pd.Series(range(365))

In [24]:
bd

0        0
1        1
2        2
3        3
4        4
      ... 
360    360
361    361
362    362
363    363
364    364
Length: 365, dtype: int64

In [26]:
t = bd.sample(23, replace=True)

In [27]:
t.duplicated()

133    False
212    False
53     False
217    False
198    False
316    False
11     False
0      False
109    False
21     False
173    False
123    False
125    False
149    False
285    False
134    False
139    False
149     True
262    False
132    False
226    False
29     False
23     False
dtype: bool

In [28]:
t.duplicated().max()

True

In [42]:
rooms = [bd.sample(23, replace=True).duplicated().max() for _ in range(10000)]

In [43]:
rooms[:10]

[True, True, True, True, True, True, False, True, True, True]

In [44]:
np.mean(rooms)

0.5063

Экзамен проходит по следующей схеме: если некоторый билет уже был вытянут, то после ответа экзаменатор откладывает его в сторону. Студент выучичил 20 вопросов из 30. Когда ему выгоднее идти, первым или вторым, что вероятность вытянуть выученный билет была больше?

In [45]:
tickets = list(range(1, 31))

In [46]:
student = list(range(1, 21))

In [48]:
from random import shuffle

In [49]:
shuffle(tickets)

In [61]:
n = 10000
student = list(range(1, 21))
tickets = list(range(1, 31))
result = []
for _ in range(n):
  shuffle(tickets)
  result.append(tickets[0] in student)
  # result.append(tickets[1] in student)

In [62]:
np.mean(result)

0.663

In [63]:
# Ответ: всегда вер. будет 2/3

Ночью произошло ДТП с участием такси. В городе работают две компании такси - Синие и Зеленые. Зеленым принадлежат 85% такси, Синим - 15%. Свидетель аварии утверждает, что такси принадлежало Синим. Следственный эксперимент показал, что ночью свидетель верно определяет цвет такси в 80% случаев. Какова вероятность того, что такси действительно принадлежало Синим?

In [64]:
np.random.binomial(1, 0.15)

0

In [74]:
def witness(taxi):
  if np.random.binomial(1, 0.8):
    return taxi
  return abs(taxi - 1)

In [75]:
witness(0)

1

In [82]:
n = 100000
result = []
for _ in range(n):
  taxi = np.random.binomial(1, 0.15)
  witness_answer = witness(taxi)
  result.append((taxi, witness_answer))

In [83]:
t = pd.DataFrame(result, columns=['taxi', 'witness_answer'])

In [84]:
t

,taxi,witness_answer
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
99995,1,1
99996,0,0
99997,0,0
99998,0,1


In [85]:
t.groupby('witness_answer')['taxi'].mean()

witness_answer
0    0.042560
1    0.414327
Name: taxi, dtype: float64

In [86]:
 # Ответ: 41%

Русская рулетка. Есть револьвер с 2 вставленными подряд патронами. Первый крутит барабан, стреляет и остаётся жив. Твоя очередь: тебе предлагает покрутить барабан перед выстрелом. Согласишься или сразу выстрелишь и почему?

In [87]:
chamber = [1, 1, 0, 0, 0, 0]

In [89]:
def one_turn(chamber):
  chamber.insert(0, chamber.pop())
  return chamber

In [111]:
def spin_chamber(chamber):
  n = np.random.randint(0, 6)
  for _ in range(n):
    chamber = one_turn(chamber)
  return chamber  

In [115]:
pd.Series([spin_chamber([1, 1, 0, 0, 0, 0]) for _ in range(10000)]).astype(str).value_counts(normalize=True)

[0, 0, 0, 1, 1, 0]    0.1690
[1, 0, 0, 0, 0, 1]    0.1688
[0, 1, 1, 0, 0, 0]    0.1670
[0, 0, 1, 1, 0, 0]    0.1669
[0, 0, 0, 0, 1, 1]    0.1663
[1, 1, 0, 0, 0, 0]    0.1620
dtype: float64

In [118]:
# заряжаем револьвер
chamber = [1, 1, 0, 0, 0, 0]
# первый игрок крутит барабан
chamber = spin_chamber(chamber)
# первый игрок стреляет
chamber = one_turn(chamber)
p1 = chamber[0]
chamber[0] = 0
# второй игрок вращает барабан -- нужно ли это действие?
chamber = spin_chamber(chamber)
# второй игрок стреляет
chamber = one_turn(chamber)
p2 = chamber[0]

In [119]:
p1

1

In [120]:
p2

0

In [127]:
n = 10000
result = []

for _ in range(n):
  # заряжаем револьвер
  chamber = [1, 1, 0, 0, 0, 0]
  # первый игрок крутит барабан
  chamber = spin_chamber(chamber)
  # первый игрок стреляет
  chamber = one_turn(chamber)
  p1 = chamber[0]
  chamber[0] = 0
  # # второй игрок вращает барабан -- нужно ли это действие?
  # chamber = spin_chamber(chamber)
  # второй игрок стреляет
  chamber = one_turn(chamber)
  p2 = chamber[0]

  result.append((p1, p2))

In [128]:
t = pd.DataFrame(result, columns=['p1', 'p2'])

In [129]:
t.groupby('p1')['p2'].agg(['count', 'mean']) # шансы умереть

,count,mean
p1,,
0,6579,0.25399
1,3421,0.48670


In [126]:
# Ответ: выгоднее не вращать барабан